<a href="https://colab.research.google.com/github/saisampathkumar/AI-Cybersecurity/blob/master/ICP8/L5_Encrypted_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [1]:
!pip install syft

     |████████████████████████████████| 225kB 4.9MB/s 
     |████████████████████████████████| 1.4MB 46.8MB/s 
     |████████████████████████████████| 81kB 28.6MB/s 
     |████████████████████████████████| 389kB 39.9MB/s 
     |████████████████████████████████| 450kB 43.4MB/s 
     |████████████████████████████████| 204kB 50.0MB/s 
     |████████████████████████████████| 256kB 51.2MB/s 
     |████████████████████████████████| 276kB 48.6MB/s 
     |████████████████████████████████| 51kB 21.7MB/s 
     |████████████████████████████████| 122kB 46.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [2]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0709 23:05:30.098238 139708468901760 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 23:05:30.113482 139708468901760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0709 23:05:34.636977 139708468901760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:34.638823 139708468901760 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:34.640483 139708468901760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:34.641966 139708468901760 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:34.643997 139708468901760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:34.646017 139708468901760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [7]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


Bob loss 0.10303328186273575
alice loss 0.39058154821395874
Bob loss 0.07168364524841309
alice loss 0.14354634284973145
Bob loss 0.05343588441610336
alice loss 0.06807781010866165
Bob loss 0.04241501912474632
alice loss 0.0435592345893383
Bob loss 0.035413049161434174
alice loss 0.03429843485355377
Bob loss 0.030675768852233887
alice loss 0.029722686856985092
Bob loss 0.027241665869951248
alice loss 0.026690155267715454
Bob loss 0.02458159253001213
alice loss 0.024258077144622803
Bob loss 0.02240241877734661
alice loss 0.022135263308882713
Bob loss 0.020539969205856323
alice loss 0.02022426947951317


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [9]:
secure_worker._objects # can you identify those objects in the Secure worker?

{27267607396: Parameter containing:
 tensor([-0.1306], requires_grad=True), 45985112335: Parameter containing:
 tensor([[0.9779, 0.2696]], requires_grad=True), 50360798975: Parameter containing:
 tensor([[0.6799, 0.2611]], requires_grad=True), 62056513928: Parameter containing:
 tensor([-0.1737], requires_grad=True)}

### Step 6: Average The Models

In [10]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([-0.1521])

### Step 7: The complete solution over multiple training sessions

In [11]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0436) Alice:tensor(0.0435)
Bob:tensor(0.0186) Alice:tensor(0.0228)
Bob:tensor(0.0077) Alice:tensor(0.0102)
Bob:tensor(0.0032) Alice:tensor(0.0045)
Bob:tensor(0.0013) Alice:tensor(0.0020)
Bob:tensor(0.0005) Alice:tensor(0.0009)
Bob:tensor(0.0002) Alice:tensor(0.0004)
Bob:tensor(6.0806e-05) Alice:tensor(0.0002)
Bob:tensor(1.8346e-05) Alice:tensor(7.4525e-05)
Bob:tensor(4.4486e-06) Alice:tensor(3.3498e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [13]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0055],
        [0.0105],
        [0.9853],
        [0.9903]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0001)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [15]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [16]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [17]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [18]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [19]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [21]:
shares = encrypt(x=7, n_share=10)
shares

(14984409589156104790992,
 4908993016761646595095,
 2083858776495110910803,
 16596287534363797546180,
 584367929445028451446,
 13557693407698941189353,
 3719018774562839278994,
 696857507475208834704,
 12679733745225780276148,
 1410669250096260586461)

In [22]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [24]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(23421469915321777368899, 7656951558160301090942, 16402838214038400513610)
(20227809385917007168047, 11611825479799997058374, 15641624821803474747032)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(-0.600)

23740629843760239345664

In [28]:
decode(9323)

9.323

In [29]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (11321590865232676678907, 8342858144251255043545, 4076180834276307769771)
Value of y = (20140630538466347559603, 8487915402051803248394, 18852713747002328170949)
Value of sum = (7721591559938784751787, 16830773546303058291939, 22928894581278635940720)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [30]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0709 23:05:35.967966 139708468901760 hook.py:98] Torch was already hooked... skipping hooking process
W0709 23:05:35.969637 139708468901760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:35.971398 139708468901760 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:35.972476 139708468901760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:35.974319 139708468901760 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:35.975623 139708468901760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 23:05:35.977440 139708468901760 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:44>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [32]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:85836912231 -> bob:38138198048]
	-> (Wrapper)>[PointerTensor | me:32974527960 -> alice:41844078997]
	-> (Wrapper)>[PointerTensor | me:93279030215 -> secure_worker:79870982249]
	*crypto provider: me*

In [33]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{17330579554: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 25090084529: tensor([[-0.0013],
         [ 0.0017]], grad_fn=<AddmmBackward>), 38138198048: tensor([3368861942271332473, 2478475509059797014, 3268599672746412702,
         1362355627352659748, 2531391821424818151]), 90279360527: tensor([[0.],
         [0.]], requires_grad=True)}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [34]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:80158234639 -> bob:94421470115]
	-> (Wrapper)>[PointerTensor | me:61228282398 -> alice:17351665857]
	-> (Wrapper)>[PointerTensor | me:23605620320 -> secure_worker:54591124730]
	*crypto provider: me*

In [35]:
bob._objects # bob has another tensor.. why?

{17330579554: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 25090084529: tensor([[-0.0013],
         [ 0.0017]], grad_fn=<AddmmBackward>), 38138198048: tensor([3368861942271332473, 2478475509059797014, 3268599672746412702,
         1362355627352659748, 2531391821424818151]), 90279360527: tensor([[0.],
         [0.]], requires_grad=True), 94421470115: tensor([6737723884542664946, 4956951018119594028, 6537199345492825404,
         2724711254705319496, 5062783642849636302])}

In [36]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [38]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [39]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [40]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [41]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [42]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [43]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [45]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:30167584162 -> bob:31333935836]
	-> (Wrapper)>[PointerTensor | me:31340638784 -> alice:71445694794]
	*crypto provider: me*

In [46]:
bob._objects

{17330579554: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 25090084529: tensor([[-0.0013],
         [ 0.0017]], grad_fn=<AddmmBackward>), 31333935836: tensor([1492145717177087256, 3677557290136101338,  458070370911237219]), 38138198048: tensor([3368861942271332473, 2478475509059797014, 3268599672746412702,
         1362355627352659748, 2531391821424818151]), 90279360527: tensor([[0.],
         [0.]], requires_grad=True)}

In [0]:
y = x + x

In [48]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [96]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0710 00:41:57.382650 139708468901760 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create cells as needed. Separate your solution into logical cells.

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [98]:
bob.add_workers([alice, ted, carol, secure_worker])
alice.add_workers([bob, ted, carol, secure_worker])
ted.add_workers([alice, bob, carol, secure_worker])
carol.add_workers([alice, bob, ted, secure_worker])
secure_worker.add_workers([alice, bob, ted, carol])


W0710 00:42:00.119156 139708468901760 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.121250 139708468901760 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.123243 139708468901760 base.py:628] Worker carol already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.127120 139708468901760 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.129045 139708468901760 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.129984 139708468901760 base.py:628] Worker ted already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:42:00.131710 139708468901760 base.p

<VirtualWorker id:secure_worker #objects:61>

In [119]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.],[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.],[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:4].send(alice)
alices_target = target[2:4].send(alice)

teds_data = data[4:6].send(ted)
teds_target = target[4:6].send(ted)

carols_data = data[6:].send(carol)
carols_target = target[6:].send(carol)

(Wrapper)>[PointerTensor | me:30997021059 -> alice:80109282804]


In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)


In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 
teds_model = model.copy().send(ted)
carols_model = model.copy().send(carol)


bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
teds_opt = optim.SGD(params=teds_model.parameters(), lr=0.1)
carols_opt = optim.SGD(params=carols_model.parameters(), lr=0.1)

In [102]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')
    
    # Train ted's Model
    teds_opt.zero_grad()
    teds_pred = teds_model(teds_data)
    teds_loss = ((teds_pred - teds_target)**2).mean()
    teds_loss.backward()

    teds_opt.step()
    teds_loss = teds_loss.get().data
    print(f'ted loss {teds_loss}')
    
    # Train carol's Model
    carols_opt.zero_grad()
    carols_pred = carols_model(carols_data)
    carols_loss = ((carols_pred - carols_target)**2).mean()
    carols_loss.backward()

    carols_opt.step()
    carols_loss = carols_loss.get().data
    print(f'carol loss {carols_loss} \n')

Bob loss 0.2979990839958191
alice loss 2.351238250732422
ted loss 0.2979990839958191
carol loss 2.351238250732422 

Bob loss 0.165480375289917
alice loss 0.7096378803253174
ted loss 0.165480375289917
carol loss 0.7096378803253174 

Bob loss 0.09303484112024307
alice loss 0.2228870689868927
ted loss 0.09303484112024307
carol loss 0.2228870689868927 

Bob loss 0.0533425435423851
alice loss 0.07780726999044418
ted loss 0.0533425435423851
carol loss 0.07780726999044418 

Bob loss 0.03151467815041542
alice loss 0.0338781513273716
ted loss 0.03151467815041542
carol loss 0.0338781513273716 

Bob loss 0.019436601549386978
alice loss 0.019953472539782524
ted loss 0.019436601549386978
carol loss 0.019953472539782524 

Bob loss 0.012685254216194153
alice loss 0.014983044937252998
ted loss 0.012685254216194153
carol loss 0.014983044937252998 

Bob loss 0.00884933304041624
alice loss 0.012734074145555496
ted loss 0.00884933304041624
carol loss 0.012734074145555496 

Bob loss 0.006613883189857006
al

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)
teds_model.move(secure_worker)
carols_model.move(secure_worker)

In [90]:
secure_worker._objects # can you identify those objects in the Secure worker?

{795710790: Parameter containing:
 tensor([0.0798], requires_grad=True), 4804221421: Parameter containing:
 tensor([0.5997], requires_grad=True), 5860764373: Parameter containing:
 tensor([-0.0011], requires_grad=True), 7378777253: Parameter containing:
 tensor([-0.0101], requires_grad=True), 8638388743: Parameter containing:
 tensor([[0.9714, 0.0055]], requires_grad=True), 8952680923: Parameter containing:
 tensor([-0.2317], requires_grad=True), 9052160478: Parameter containing:
 tensor([[0.9470, 0.0282]], requires_grad=True), 9469215951: Parameter containing:
 tensor([0.0066], requires_grad=True), 11236359287: Parameter containing:
 tensor([0.0108], requires_grad=True), 11493416897: Parameter containing:
 tensor([-0.0273], requires_grad=True), 12018743972: Parameter containing:
 tensor([[0.9422, 0.1296]], requires_grad=True), 14389427897: Parameter containing:
 tensor([0.3797], requires_grad=True), 15527979550: Parameter containing:
 tensor([0.0798], requires_grad=True), 18789838339:

In [104]:
model.weight.data[0].set_(((carols_model.weight.data + teds_model.weight.data + alices_model.weight.data + bobs_model.weight.data) / 4).get())
model.bias.data[0].set_(((carols_model.weight.data + teds_model.weight.data + alices_model.weight.data + bobs_model.weight.data) / 4).get())

tensor([[0.3583, 0.1492]])

In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()


In [106]:
print(preds)
print("\n",target)
print("\n",loss.data)

tensor([[-0.5088],
        [-0.5807],
        [-0.4970],
        [-0.5689],
        [-0.5088],
        [-0.5807],
        [-0.4970],
        [-0.5689]], grad_fn=<AddmmBackward>)

 tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)

 tensor(1.3246)


In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

In [109]:
x = x.share(bob, alice, ted, carol, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:93959091820 -> bob:40132313161]
	-> (Wrapper)>[PointerTensor | me:1497542042 -> alice:75554133453]
	-> (Wrapper)>[PointerTensor | me:97760606986 -> ted:96606130684]
	-> (Wrapper)>[PointerTensor | me:72578491105 -> carol:49484385255]
	-> (Wrapper)>[PointerTensor | me:3737510475 -> secure_worker:16172785571]
	*crypto provider: me*

In [110]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{12696864914: tensor([[0.],
         [0.]], requires_grad=True), 18214986091: Parameter containing:
 tensor([0.2381], requires_grad=True), 25111751606: tensor([[-0.1025],
         [ 0.0051]], grad_fn=<AddmmBackward>), 26934830907: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 31333935836: tensor([1492145717177087256, 3677557290136101338,  458070370911237219]), 38138198048: tensor([3368861942271332473, 2478475509059797014, 3268599672746412702,
         1362355627352659748, 2531391821424818151]), 39822807410: Parameter containing:
 tensor([[-0.2586, -0.5000,  0.4388,  0.0910]], requires_grad=True), 40132313161: tensor([4428375160682429082, 1357404699281054040, 4109406651830394048,
         1945019101042711838,  294734790974642204]), 65765541185: tensor([[0., 0.],
         [0., 1.]], requires_grad=True)}

In [111]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:65218290209 -> bob:1838052896]
	-> (Wrapper)>[PointerTensor | me:14538281318 -> alice:17190622709]
	-> (Wrapper)>[PointerTensor | me:173358419 -> ted:4657610747]
	-> (Wrapper)>[PointerTensor | me:56545111912 -> carol:69581252830]
	-> (Wrapper)>[PointerTensor | me:80566257167 -> secure_worker:94988621610]
	*crypto provider: me*

In [112]:

bob._objects # bob has another tensor.. why?

{1838052896: tensor([8856750321364858164, 2714809398562108080, 8218813303660788096,
         3890038202085423676,  589469581949284408]), 12696864914: tensor([[0.],
         [0.]], requires_grad=True), 18214986091: Parameter containing:
 tensor([0.2381], requires_grad=True), 25111751606: tensor([[-0.1025],
         [ 0.0051]], grad_fn=<AddmmBackward>), 26934830907: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 31333935836: tensor([1492145717177087256, 3677557290136101338,  458070370911237219]), 38138198048: tensor([3368861942271332473, 2478475509059797014, 3268599672746412702,
         1362355627352659748, 2531391821424818151]), 39822807410: Parameter containing:
 tensor([[-0.2586, -0.5000,  0.4388,  0.0910]], requires_grad=True), 40132313161: tensor([4428375160682429082, 1357404699281054040, 4109406651830394048,
         1945019101042711838,  294734790974642204]), 65765541185: tensor([[0., 0.],
         [0., 1.]], requires_grad=True)}

In [113]:

y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])